In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()
from tdmclient import ClientAsync, aw
client = ClientAsync()
client.process_waiting_messages()
node = await client.wait_for_node()
aw(node.run())
aw(node.stop())
aw(node.unlock())

import time
from l42 import *
from local_nav import *
from VisGraph import *
from update_pos import *


In [2]:
#getters and setters functions

@tdmclient.notebook.sync_var
def get_prox_value():
    global prox_horizontal
    return prox_horizontal

#@tdmclient.notebook.sync_var
#def get_ground_value():
#    global prox_ground_reflected
#    return prox_ground_reflected

@tdmclient.notebook.sync_var
def get_left_speed():
    global motor_left_target
    return motor_left_target

@tdmclient.notebook.sync_var
def get_right_speed():
    global motor_right_target
    return motor_right_target

@tdmclient.notebook.sync_var
def set_motor_speed(left_speed, right_speed):
    global motor_left_target, motor_right_target
    motor_left_target = left_speed
    motor_right_target = right_speed

In [3]:
#Create constant variables
next_goal_trigger = 20

#Creation of the map and the global path
global_path = initialisation()
start = global_path[0]
stop = global_path[len(global_path)-1]

#Set the initial motor speeds at 0
set_motor_speed(0, 0)

#Initialisation of the variables
prox_sensors = np.array([0, 0, 0, 0, 0, 0, 0])

#Find the initial position and orientation of the robot
position, orientation = detect_thymio()

#Initialisation of the variables that will keep in memory the position and orientation of the last iteration
old_position = position
old_orientation = orientation

#Initialisation of the variables that will keep track of the current goal position and orientation
current_goal = 1
current_goal_pos = global_path[current_goal]
print(position)

distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

#Initialisation of the variables that will be updated to set the speed of the wheels
left_speed = 0
right_speed = 0

#Initialisation of the boolean that will recall if the camera detection is possible or not at each iteration
possible_detection = True

while(True):
    print(current_goal)
    current_goal_pos = global_path[current_goal]
    #Recall the values of the sensors and the wheel speeds 
    prox_sensors = get_prox_value()
    #prox_ground_values = get_ground_value()
    #set_motor_speed(100, 100)
    
    #Reset and update the detection boolean
    #possible_detection = #fonction prévue à cet effet
    
    #if possible_detection :
        #Vision : Returns the position and the orientation of the robot from the camera
    position, orientation = detect_thymio()
    print(position)
    
    #else :
        #The position and orientation of the robot is estimated from its last state and the wheel speeds
        #position = kalman(position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction 
        #orientation = kalman(old_position, orientation, left_speed, right_speed) #Ou peu importe la forme de la fonction
    
    #Update the current goal orientation and the distance from the robot
    distance_to_goal, current_goal_orientation = calculation_distance_and_angle(position, current_goal_pos)

    #Find the speeds to apply to the robot
    left_speed, right_speed = localNavigation(orientation, old_orientation, current_goal_orientation, prox_sensors)
    #Apply the desired speeds to the robot
    set_motor_speed(int(left_speed), int(right_speed))
    
    #Keep in memory the position and orientation of the robot for the next 
    old_position = position
    old_orientation = orientation
    
    #Update the current goal if necessary
    if (distance_to_goal < next_goal_trigger) :
        current_goal += 1
    
    if (current_goal == len(global_path)) :
        set_motor_speed(0, 0)
        break
    
    await client.sleep(0.1)
  




No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...
No detection...


100%|██████████████████████████████████████████| 2/2 [00:00<00:00, 39.30it/s]


No thymio
No thymio
No thymio
No thymio
[59 12]
1
No thymio
No thymio
No thymio
[61 12]
1
No thymio
No thymio
[64 12]
1
No thymio
No thymio
No thymio
[69  1]
1
No thymio
No thymio
No thymio
No thymio
No thymio
[60 21]
1
No thymio
[67 16]
1
No thymio
No thymio
No thymio
[69 23]
1
No thymio
No thymio
[74 26]
1
No thymio
[71 38]
1
No thymio
No thymio
No thymio
[68 33]
1
No thymio
[70 38]
1
No thymio
No thymio
[76 42]
1
No thymio
No thymio
No thymio
[69 45]
1
No thymio
[71 47]
1
No thymio
[76 53]
1
No thymio
[76 55]
1
No thymio
No thymio
No thymio
[74 56]
1
No thymio
[79 65]
1
No thymio
[84 69]
1
No thymio
[84 72]
1
No thymio
[86 75]
1
No thymio
[84 78]
1
No thymio
No thymio
[88 82]
1
No thymio
[87 85]
1
No thymio
[90 90]
1
No thymio
[92 93]
1
No thymio
[91 95]
1
No thymio
[ 94 100]
1
No thymio
[ 95 102]
1
No thymio
[ 96 106]
1
No thymio
[ 99 109]
1
No thymio
[100 113]
1
No thymio
[102 116]
1
No thymio
[104 119]
1
No thymio
[106 122]
1
No thymio
[107 126]
1
No thymio
[109 130]
1
No thymio


D:\EPFL\MA1\Basics of mobile robotics\Team-yo\local_nav.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  angle = np.around(np.arctan((position_goal[1] - position_thymio[1]) / (position_goal[0] - position_thymio[0])), 3)


3
No thymio
[342 307]
3
No thymio
[345 312]
3
No thymio
[348 316]
3
No thymio
[349 321]
4
No thymio
[351 325]
4
No thymio
[353 330]
4
No thymio
[354 335]
4
No thymio
[355 339]
4
No thymio
[356 343]
4
No thymio
[358 348]
4
No thymio
[360 353]
4
No thymio
[361 357]
4
No thymio
[364 361]
4
No thymio
[366 366]
4
No thymio
[369 370]
4
No thymio
[371 375]
4
No thymio
[374 379]
4
No thymio
[377 382]
4
No thymio
[381 387]
4
No thymio
[384 390]
4
No thymio
[387 392]
4
No thymio
[391 396]
4
No thymio
[394 399]
4
No thymio
[398 402]
4
No thymio
[404 404]
5
No thymio
[408 407]
5
No thymio
[412 408]
5
No thymio
[417 409]
5
No thymio
[421 411]
5
No thymio
[426 412]
5
No thymio
[432 414]
5
No thymio
[436 414]
5
No thymio
[442 416]
5
No thymio
[446 417]
5
No thymio
[451 418]
5
No thymio
No thymio
[454 421]
5
No thymio
[461 421]
5
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No thymio
No t

5
No thymio
[494 284]
5
No thymio
[494 288]
5
No thymio
[494 292]
5
No thymio
[494 298]
5
No thymio
[494 301]
5
No thymio
[494 307]
5
No thymio
[493 311]
5
No thymio
[493 317]
5
No thymio
[493 321]
5
No thymio
[493 326]
5
No thymio
[493 331]
5
No thymio
[493 336]
5
No thymio
[493 339]
5
No thymio
[492 344]
5
No thymio
[492 349]
5
No thymio
[492 355]
5
No thymio
[492 358]
5
No thymio
[491 363]
5
No thymio
[491 368]
5
No thymio
[491 372]
5
No thymio
[490 378]
5
No thymio
[490 383]
5
No thymio
[490 388]
5
No thymio
[490 392]
5
No thymio
[489 397]
5
No thymio
[489 402]
5
No thymio
[489 408]
5
No thymio
[488 412]


In [4]:
await tdmclient.notebook.stop()